### Getting docs

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Indexing data

In [2]:
from elasticsearch import Elasticsearch

es_url = 'http://127.0.0.1:9200/'

es_client = Elasticsearch(es_url)

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'homework-index'

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_29795/1970897794.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-index'})

In [6]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████| 948/948 [00:02<00:00, 340.04it/s]


### Querying data

In [33]:
# query= "How do execute a command on a Kubernetes pod?"
query= "How do copy a file to a Docker container?"

In [34]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        # field question is 3 times more important than the other ones
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [35]:
res = elastic_search(query)

/tmp/ipykernel_29795/1291232902.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


## Building a prompt

In [38]:
question = "How do I execute a command in a running docker container?"

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for entry in res:
    context += f"{context_template.format(question=entry['question'], text=entry['text'])}\n\n"



context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker 

In [39]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=question, context=context)

### Tokens

In [54]:
from google import genai
import os
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]


client = genai.Client(api_key=GEMINI_API_KEY)

total_tokens = client.models.count_tokens(model="gemini-2.0-flash", contents=prompt)

In [56]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

In [58]:
response.text

'If the container is already running, first find the container-id using `docker ps`. Then, execute your command in the container using `docker exec -it <container-id> bash`.\n'